### Poisson Distribution: Predict the score in soccer betting
- How to use Poisson Distribution to predict soccer scores
- Using Defence Strength & Attack Strength values
- Calculate the most likely score-line
- Converting estimated chance into odds

### Calculate Attack Strength
The ratio of a team's average and the league average is what constitutes “Attack Strength”.

### Ler arquivo

In [470]:
import pandas as pd
dfx = pd.read_csv("./arquivos/Italy/italy_geral_computed_indices.csv", parse_dates=["Date"]).reset_index(drop=True)

In [471]:
# Season for calc the strenght
SEASON = '2017/2018'

In [472]:
df = dfx[dfx['SEASON'] == SEASON].sort_values('Date', ascending=True)

In [473]:
# p = int(len(dfx)/10)
df = df.reset_index(drop=True)

League ratio of a team's average and the league average

In [474]:
# League average home

LAH = df[df['SEASON'] == SEASON]['FTHG'].sum()/df[df['SEASON'] == SEASON]['FTHG'].count()


In [475]:
# Team average home

TAH = df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTHG'].sum()/df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTHG'].count()

In [476]:
# Team attack strenght home
ASH = {k: v['FTHG'] for k, v in pd.DataFrame(TAH/LAH).iterrows()}

In [477]:
# League average Away

LAA = df[df['SEASON'] == SEASON]['FTAG'].sum()/df[df['SEASON'] == SEASON]['FTAG'].count()


In [478]:
# Team average Away

TAA = df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTAG'].sum()/df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTAG'].count()

In [479]:
# Team attack strenght Away
ASA = {k: v['FTAG'] for k, v in pd.DataFrame(TAA/LAA).iterrows()}

### Calculate Defence Strength

In [480]:
# Team average conceived goals at Home

TACGH = df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTAG'].sum()/df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTAG'].count()


In [481]:
# Defense Strenght Home
DSH = {k: v['FTAG'] for k, v in pd.DataFrame(TACGH/LAA).iterrows()}

In [482]:
# Team average conceived goals Away
TACGA = df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTHG'].sum()/df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTHG'].count()


In [483]:
# Defense Strenght Away
DSA = {k: v['FTHG'] for k, v in pd.DataFrame(TACGA).iterrows()}

### We can now use the following formula to calculate the likely number of goals a team might score

In [484]:
# Home Score
def calc_home_score(H, A):
    return ASH[H] * DSA[A] * LAH

In [485]:
# Calc away score
def calc_away_score(H, A):
    return ASA[A] * DSH[H] * LAA

In [530]:
def predict(X):
    y = []
    for x in X:
        
        home_score = calc_home_score(x[0], x[1])
        away_score = calc_away_score(x[0], x[1])
        
        if abs(home_score - away_score) < home_score * 0.05:
            y.append('D')
        else:
        
            if  home_score > away_score:
                y.append('H')
            else:
                y.append('A')
            
    return y

In [531]:
dft = dfx

In [532]:
# Prepare the data sets
X, y_true = dft[dft['SEASON'] == '2018/2019'][['HomeTeam','AwayTeam']].values, dft[dft['SEASON'] == '2018/2019']['FTR'].values

In [533]:
y_pred = predict(X)

### Avaliar precisão

In [534]:
r = []
for i in range(len(y_pred)):   
    if y_pred[i] == y_true[i]:        
        r.append(1)
    else:
        r.append(0)   
    

In [535]:
np.array(r).mean()

0.5234375